In [ ]:
print("all good");

In [ ]:
from langchain_groq import ChatGroq
from dotenv import load_dotenv
load_dotenv()

In [ ]:
model=ChatGroq(
    model="qwen/qwen3-32b")
model.invoke("What is the capital of France?").content

In [ ]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
embedding=GoogleGenerativeAIEmbeddings(model="models/embedding-001").embed_query("What is the capital of France?")
print(embedding)    

In [ ]:
# Data ingestion to vector store
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

import os
filepath=os.path.join(os.getcwd(), "data", "sample.pdf")

In [ ]:
#PyPDFLoader(filepath)
loader=PyPDFLoader(filepath)
documents=loader.load()
#RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200,length_function=len)
split_docs=text_splitter.split_documents(documents)

len(split_docs)
print(split_docs[0].page_content)
print(split_docs[0].metadata)


In [ ]:
# store in vector store
from langchain_community.vectorstores import FAISS


In [ ]:
vectorstore=FAISS.from_documents(
    documents=split_docs,
    embedding=GoogleGenerativeAIEmbeddings(model="models/embedding-001"))

In [ ]:

relavancedocument=vectorstore.similarity_search("llama2 finetuning benchmark experiments.",k=2)
for doc in relavancedocument:
    print(f'''{doc.metadata['page']}->{doc.page_content} \n''' )
    

In [ ]:
reteiver=vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 2})
reteiver.invoke("llama2 finetuning benchmark experiments.")

In [ ]:
prompt_template = """
        Answer the question based on the context provided below. 
        If the context does not contain sufficient information, respond with: 
        "I do not have enough information about this."

        Context: {context}

        Question: {question}

        Answer:"""

In [ ]:
from langchain.prompts import PromptTemplate
prompt=PromptTemplate(
    template=prompt_template,
    input_variables=["context", "question"]
)

In [ ]:
prompt

In [ ]:
from langchain_core.output_parsers import StrOutputParser
parser=StrOutputParser()

In [ ]:
def format_docs(docs):
    return "\n\n".join([doc.page_content for doc in docs])

from langchain_core.runnables import RunnablePassthrough
rag_chain = (
    {"context": reteiver | format_docs, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [ ]:
rag_chain.invoke("tell  me about the llama2 finetuning benchmark experiments?")